<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2022Q2/Kaggle_2Q2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de datos

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

In [3]:
import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle competitions download -c ap-tp2-2q2022


import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
100% 51.3M/51.3M [00:00<00:00, 84.1MB/s]
100% 51.3M/51.3M [00:00<00:00, 71.4MB/s]


In [4]:
db=pd.read_csv("origen.csv", sep=",")

In [6]:
db.columns

Index(['Id', 'averageRating', 'numVotes', 'titleType', 'isAdult', 'startYear',
       'endYear', 'runtimeMinutes', 'genres_x', 'directors', 'writers',
       'seasonNumber', 'episodeNumber', 'ordering', 'language', 'attributes',
       'isOriginalTitle', 'adult', 'budget', 'genres_y', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime', 'status',
       'tagline', 'video'],
      dtype='object')

# Regresión lineal

In [7]:
X=['numVotes','startYear','runtimeMinutes']

In [8]:
reg = LinearRegression().fit(db[X], db['averageRating'])
reg.score(db[X], db['averageRating'])

0.024976875345017358

# Salida

In [9]:
res=pd.read_csv("testear.csv", sep=",")

In [10]:
resultado=reg.predict(res[X])

In [11]:
salida = pd.DataFrame(data={"averageRating": resultado}).astype(str)
salida.index = res.Id
salida.to_csv("Linear_Reg.csv", sep=',',index=True,  index_label='Id')

# Extra Trees

In [12]:
from sklearn.ensemble import ExtraTreesRegressor

In [13]:
reg = ExtraTreesRegressor().fit(db[X], db['averageRating'])
reg.score(db[X], db['averageRating'])

0.5067633418019462

# Train Test Split

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(db[X], db['averageRating'], test_size=0.2, random_state=10)

In [16]:
reg = ExtraTreesRegressor().fit(X_train, y_train)
reg.score(X_test,  y_test)

0.06332556847731052

# Regularizar

In [17]:
reg = ExtraTreesRegressor(max_depth=20).fit(X_train, y_train)
reg.score(X_test,  y_test)

0.16308167729176037

¿Qué sigue? ¿Cómo mejorar? 

- Hicimos una regularización y el modelo mejoró, pero eso no quiere decir que el modelo esté optimizado. Habría que seguir probando, incluso convendría hacerlo por [Cross Validation](https://scikit-learn.org/stable/modules/cross_validation.html). 
- Solo regularizamos con max_depth. [Extra Trees](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html) tiene otros hiperparámetros que podrían mejorar la performance. Conocer los modelos ayuda a tener mejor intuición de que es lo que puede pasar al tocar cada hiperparámetro, pero nada me garantiza resultados: hay que probar. 
- Probar a mano (grid o random search) puede llevar mucho tiempo. Por eso existen técnicas, como por ejemplo la [optimización bayesiana](https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a). 
- Probamos con Extra Trees que es uno de los modelos que mejor funcionan. Sin embargo, hay muchos otros. Casi con seguridad [Catboost](https://catboost.ai/) y [XGBoost](https://xgboost.readthedocs.io/en/stable/) funcionan mejor. Hay que probarlos. 
- Todavía el data set tiene mucha información para incorporar. ¡Estamos usando solo tres features! Hay que limpiar e incorporar las demás. Por más que usemos el mejor modelo y esté optimizado de la mejor manera, tal vez con solo tres features no logremos pasar el 0.20 en testeo. 
- Hay que hacer EDA, Data Wrangling, Data Discovery. 
- ¿Por qué limitarse a los datos que se presentan? Por ejemplo, tal vez los directores más jóvenes tienen mejor rating. ¿Cómo puedo incorporar esa información? 
- También puedo forzar relaciones entre las features, especialmente si creo que pueden tener un significado para la explicación de los datos. Esto es parte de lo que se conoce como [feature engineering](https://www.projectpro.io/article/8-feature-engineering-techniques-for-machine-learning/423). 
- En esta línea podría ayudar alguna técnia de [reducción de la dimensionalidad](https://wikipedia.org/en/Dimensionality_reduction). O [Shapley Values](https://towardsdatascience.com/the-shapley-value-for-ml-models-f1100bff78d1), [PDP](https://christophm.github.io/interpretable-ml-book/pdp.html), [Ale Plots](https://christophm.github.io/interpretable-ml-book/ale.html), etc. 
- Una vez que tengamos un score de testeo que nos satisfaga, todavía tenemos que ir al Kaggle y hacer la submission. Llegar a un 0.35 en testeo no implica necesariamente que en la competencia obtengamos ese resultado. Especialmente si enviciamos el modelo con mucha cross validation mal hecha. 